# ComStock Data Extraction

## Introduction
 
This notebook extracts building stock data from NREL's ComStock dataset on OEDI (Open Energy Data Initiative).
It allows you to specify states and counties of interest, downloads baseline and upgrade data,
adds county names from lookup tables, and exports the data as a CSV file. This data can be imported in the
'2. Raw Data' sheet of the "ComStock Impact Analysis Template" Microsoft Excel file. 

## 1. User Inputs
 
The first user input needed is the geography of interest as combinations of states and selected county codes. 
The geographies of interest must be input as comma-separated list of county codes (see below). To map a county 
name to the appropriate county ID for this script, please refer to the spacial_tract_lookup_table.csv file located 
at https://data.openei.org/s3_viewer?bucket=oedi-data-lake&prefix=nrel-pds-building-stock%2Fend-use-load-profiles-for-us-building-stock%2F2024%2Fcomstock_amy2018_release_2%2Fgeographic_information%2F 
Map the "resstock_county_id" column to the "nhgis_county_gisjoin" column to get the list county codes for this script.

A second input is the name and location of the output file, which the user can modify to point to a folder on their local computer.

In the following code chunk, define your geography of interest. You only need to modify this cell. After modification, you can run all cells and get the output csv.

In [6]:
# ========================
# USER CONFIGURATION
# ========================

# Specify your state(s) and county code(s) here, in the format shown below.
# Example: {'MN': ['G2700530']}
STATES_COUNTIES = {
    'MN': ['G2700530'],
    'CO': ['G0800170', 'G0800330', 'G0800990']
}

# Output CSV file path
OUTPUT_CSV = 'combined_output_test.csv'

## 2. Project Set Up and Configurations

The following code chunk loads all required libraries (make sure they are available in the current environment) and sets up the logging.
No user input required.

In [2]:
# ========================
# SET UP AND CONFIGURATION
# ========================

import polars as pl
import pandas as pd
import requests
import logging
import tempfile
import os
import time
import re
from io import StringIO
from pathlib import Path
from typing import Dict, List, Optional, Tuple, Union, Any, Set

# Configure logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s'
)
logger = logging.getLogger('OEDI_Extractor')


This code chunk defines the columns necessary for the Building Stock Characteristic, Segmentation and Upgrade Analysis 2024.2 Microsoft Excel 
workbook. The column order matters for the correct population of all plots in the workbook.
All data is available in the basic aggregate files.

The base path is the path to the current ComStock Release dataset.
The relative path from base path link to the dataset path for baseline and upgrade data retrieval,
to the lookup URL for county name matching (lookup) 
and the crosswalk URL to match an upgrade number (which may be different per ComStock release) to an upgrade_id which remains constant across releases.

In [3]:
# ========================
# SET UP AND CONFIGURATION
# ========================

# Column configuration (taken from the provided YAML)
COLUMNS = [
    'bldg_id',
    'upgrade',
    'in.upgrade_name',
    'applicability',
    'weight',
    'in.comstock_building_type_group',
    'in.floor_area_category',
    'in.hvac_category',
    'in.interior_lighting_generation',
    'in.state_name',
    'out.site_energy.total.energy_consumption_intensity..kwh_per_ft2',
    'calc.segment',
    'calc.weighted.emissions.total_with_cambium_mid_case_15y..co2e_mmt',
    'calc.weighted.emissions.total_with_egrid..co2e_mmt',
    'calc.weighted.enduse_group.district_cooling.hvac.energy_consumption..tbtu',
    'calc.weighted.enduse_group.district_heating.hvac.energy_consumption..tbtu',
    'calc.weighted.enduse_group.district_heating.water_systems.energy_consumption..tbtu',
    'calc.weighted.enduse_group.electricity.hvac.energy_consumption..tbtu',
    'calc.weighted.enduse_group.electricity.interior_equipment.energy_consumption..tbtu',
    'calc.weighted.enduse_group.electricity.lighting.energy_consumption..tbtu',
    'calc.weighted.enduse_group.electricity.refrigeration.energy_consumption..tbtu',
    'calc.weighted.enduse_group.electricity.water_systems.energy_consumption..tbtu',
    'calc.weighted.enduse_group.natural_gas.hvac.energy_consumption..tbtu',
    'calc.weighted.enduse_group.natural_gas.interior_equipment.energy_consumption..tbtu',
    'calc.weighted.enduse_group.natural_gas.water_systems.energy_consumption..tbtu',
    'calc.weighted.enduse_group.other_fuel.hvac.energy_consumption..tbtu',
    'calc.weighted.enduse_group.other_fuel.water_systems.energy_consumption..tbtu',
    'calc.weighted.enduse_group.site_energy.hvac.energy_consumption..tbtu',
    'calc.weighted.enduse_group.site_energy.interior_equipment.energy_consumption..tbtu',
    'calc.weighted.enduse_group.site_energy.lighting.energy_consumption..tbtu',
    'calc.weighted.enduse_group.site_energy.refrigeration.energy_consumption..tbtu',
    'calc.weighted.enduse_group.site_energy.water_systems.energy_consumption..tbtu',
    'calc.weighted.site_energy.total.energy_consumption..tbtu',
    'calc.weighted.sqft..ft2'
]

# Data type to use (can be 'basic' or 'full')
DATA_TYPE = 'basic'

# ========================
# CONSTANTS
# ========================
# Base URL components for aggregates at state and county level
BASE_PATH = "https://oedi-data-lake.s3.amazonaws.com/nrel-pds-building-stock/end-use-load-profiles-for-us-building-stock/2024/comstock_amy2018_release_2"
BASE_URL_TEMPLATE = f"{BASE_PATH}/metadata_and_annual_results_aggregates/by_state_and_county/{DATA_TYPE}/parquet"

# Lookup URLs for county name matching (lookup) and upgrade_id (crosswalk)
LOOKUP_URL = f"{BASE_PATH}/geographic_information/spatial_tract_lookup_table.csv"
CROSSWALK_URL = f"{BASE_PATH}/measure_name_crosswalk.csv"


## 3. Helper Functions

This section defines all helper functions to load the county lookup, load the measure crosswalk, 
construct the file URLs and download the parquet files. It also defines functions to download
and process all files into one dataset.

IMPORTANT: Some parts of the data combination rely on fallback options in case of issues. Please carefully check the logger output when running the notebook.

In [ ]:

# ========================
# HELPER FUNCTIONS
# ========================

def load_county_lookup() -> Dict[str, str]:
    """
    Load the county lookup table from the NREL GitHub repository.
    
    Returns:
        Dict[str, str]: A mapping of county codes to county names
        
    Raises:
        RuntimeError: If unable to download or process the lookup table
    """
    logger.info(f"Downloading county lookup table from {LOOKUP_URL}")
    try:
        start_time = time.time()
        response = requests.get(LOOKUP_URL)
        response.raise_for_status()
        
        # Read CSV data into pandas DataFrame
        county_lookup_df = pd.read_csv(
            StringIO(response.text),
            dtype={'nhgis_county_gisjoin': str, 'resstock_county_id': str}
        )
        
        # Create a mapping dictionary for faster lookups
        county_mapping = dict(zip(
            county_lookup_df['nhgis_county_gisjoin'], 
            county_lookup_df['resstock_county_id']
        ))
        
        logger.info(f"County lookup table loaded with {len(county_lookup_df)} entries in {time.time() - start_time:.2f} seconds")
        return county_mapping
        
    except Exception as e:
        raise RuntimeError(f"Error loading county lookup table: {str(e)}")

def load_measure_mapping() -> Tuple[Dict[str, str], Dict[str, str]]:
    """
    Load the measure name crosswalk from NREL's data lake.
    
    Returns:
        Tuple[Dict[str, str], Dict[str, str]]: 
            measure_to_upgrade_mapping and upgrade_to_measure_mapping
        
    Raises:
        RuntimeError: If unable to download or process the crosswalk
    """
    logger.info(f"Downloading measure crosswalk from {CROSSWALK_URL}")
    try:
        start_time = time.time()
        response = requests.get(CROSSWALK_URL)
        response.raise_for_status()
        
        # Read CSV data into pandas DataFrame
        measure_mapping_df = pd.read_csv(
            StringIO(response.text),
            dtype={'measure_id': str, '2024_comstock_amy2018_release_2_upgrade_id': str}
        )
        
        # Clean up the upgrade IDs to ensure proper formatting
        upgrade_col = '2024_comstock_amy2018_release_2_upgrade_id'
        
        # Fill NA values
        measure_mapping_df[upgrade_col] = measure_mapping_df[upgrade_col].fillna('')
        
        # Create mapping dictionaries for faster lookups
        measure_to_upgrade_mapping = {}
        upgrade_to_measure_mapping = {}
        
        for _, row in measure_mapping_df.iterrows():
            measure_id = row['measure_id']
            upgrade_id_raw = row[upgrade_col]
            
            # Skip empty upgrade IDs
            if pd.isna(upgrade_id_raw) or upgrade_id_raw == '':
                continue
            
            # Format upgrade ID to include leading zeros if needed
            try:
                upgrade_id_int = int(upgrade_id_raw)
                upgrade_id = f"{upgrade_id_int:02d}"
            except ValueError:
                upgrade_id = str(upgrade_id_raw)
            
            measure_to_upgrade_mapping[measure_id] = upgrade_id
            upgrade_to_measure_mapping[upgrade_id] = measure_id
        
        logger.info(f"Measure mapping loaded with {len(measure_to_upgrade_mapping)} entries in {time.time() - start_time:.2f} seconds")
        return measure_to_upgrade_mapping, upgrade_to_measure_mapping
        
    except Exception as e:
        raise RuntimeError(f"Error loading measure mapping: {str(e)}")

def construct_file_url(state: str, county: str, upgrade_id: Optional[str] = None) -> str:
    """
    Construct the URL to the parquet file for a specific state, county, and optionally upgrade.
    
    Args:
        state (str): Two-letter state code (e.g., 'CO')
        county (str): County GIS join code (e.g., 'G0800050')
        upgrade_id (Optional[str]): Upgrade ID (e.g., '03'). If None, baseline data is used.
        
    Returns:
        str: The URL to the parquet file
    """
    # Construct base URL
    base_url = BASE_URL_TEMPLATE.format(data_type=DATA_TYPE)
    
    # Determine file suffix based on data type
    file_suffix = "_agg_basic.parquet" if DATA_TYPE == 'basic' else "_agg.parquet"
    
    # Determine file name component based on upgrade_id
    if upgrade_id is None:
        file_component = "baseline"
    else:
        file_component = f"upgrade{upgrade_id}"
    
    # Construct the full URL
    return f"{base_url}/state%3D{state}/county%3D{county}/{state}_{county}_{file_component}{file_suffix}"

def download_parquet_file(url: str) -> Optional[bytes]:
    """
    Download a file from a URL.
    
    Args:
        url (str): The URL to download from
        
    Returns:
        Optional[bytes]: The file content, or None if download failed
    """
    try:
        # Download the file
        response = requests.get(url, stream=True)
        response.raise_for_status()
        
        # Read the content
        content = b''
        for chunk in response.iter_content(chunk_size=8192):
            if chunk:
                content += chunk
                
        return content
    except requests.exceptions.RequestException as e:
        logger.warning(f"Error downloading file: {str(e)}")
        return None


def download_and_process_parquet(state: str, county: str, upgrade_id: Optional[str] = None, 
                               columns: List[str] = None, county_mapping: Dict[str, str] = None) -> Optional[pl.DataFrame]:
    """
    Download and process a parquet file for a specific state/county/upgrade.
    
    Args:
        state (str): Two-letter state code
        county (str): County GIS join code
        upgrade_id (Optional[str]): Upgrade ID or None for baseline
        columns (List[str]): Columns to select
        county_mapping (Dict[str, str]): County code to name mapping
        
    Returns:
        Optional[pl.DataFrame]: Processed DataFrame or None if download failed
    """
    if columns is None:
        columns = COLUMNS
        
    file_url = construct_file_url(state, county, upgrade_id)
    upgrade_desc = f"upgrade {upgrade_id}" if upgrade_id else "baseline"
    logger.info(f"Downloading data for {state}, county {county}, {upgrade_desc} from {file_url}")
    
    try:
        start_time = time.time()
        # Create a temporary file to store the downloaded parquet
        with tempfile.NamedTemporaryFile(delete=False, suffix='.parquet') as temp_file:
            temp_filename = temp_file.name
            
            # Download the parquet file
            content = download_parquet_file(file_url)
            if content is None:
                return None
                
            # Write the content to the temporary file
            temp_file.write(content)
        
        # Read only the required columns from the parquet file
        try:
            # First check if all the requested columns exist in the file
            schema = pl.read_parquet_schema(temp_filename)
            file_columns = list(schema.keys())
            
            # Find which requested columns are actually in the file
            available_columns = [col for col in columns if col in file_columns]
            missing_columns = [col for col in columns if col not in file_columns]
            
            if missing_columns:
                logger.warning(f"The following columns were not found in the parquet file: {missing_columns}")
            
            if not available_columns:
                logger.error("None of the requested columns are available in the file")
                return None
            
            # Read only the available columns
            df = pl.read_parquet(
                temp_filename, 
                columns=available_columns
            )
            
            # Add county name column if county_mapping is provided
            if county_mapping is not None:
                if county in county_mapping:
                    county_name = county_mapping[county]
                    df = df.with_columns(pl.lit(county_name).alias("in.county_name"))
                else:
                    logger.warning(f"County code {county} not found in lookup table")
            
            logger.info(f"Successfully loaded data for {state}, county {county}, {upgrade_desc} with {len(df)} rows in {time.time() - start_time:.2f} seconds")
            return df
            
        except Exception as e:
            logger.error(f"Error reading parquet file: {str(e)}")
            return None
            
    except Exception as e:
        logger.error(f"Error processing file: {str(e)}")
        return None
    finally:
        # Clean up the temporary file
        if os.path.exists(temp_filename):
            os.unlink(temp_filename)

            
def process_all_data() -> Optional[pl.DataFrame]:
    """
    Process all counties and upgrades specified in the configuration.
    
    This function:
    1. Downloads baseline data for each county first to establish reference column types
    2. Downloads upgrade data for all upgrades in the crosswalk file
    3. Ensures column type consistency by using baseline types as reference
    4. Combines all data into a single DataFrame
    
    Returns:
        Optional[pl.DataFrame]: Combined DataFrame or None if processing failed
    """
    # Load county mapping and measure mapping
    try:
        county_mapping = load_county_lookup()
        measure_to_upgrade_mapping, upgrade_to_measure_mapping = load_measure_mapping()
    except Exception as e:
        logger.error(f"Failed to load reference data: {str(e)}")
        return None
    
    # Get all upgrade IDs to process from the crosswalk file
    upgrade_ids_to_process = list(upgrade_to_measure_mapping.keys())
    logger.info(f"Will process baseline and {len(upgrade_ids_to_process)} upgrades: {sorted(upgrade_ids_to_process)}")
    
    baseline_dataframes = []
    upgrade_dataframes = []
    
    # Step 1: Process all baseline data first to establish reference column types
    baseline_schema = None
    baseline_column_types = {}
    
    for state, counties in STATES_COUNTIES.items():
        for county in counties:
            try:
                # Download baseline data to establish reference types
                baseline_df = download_and_process_parquet(state, county, None, COLUMNS, county_mapping)
                
                if baseline_df is not None and not baseline_df.is_empty():
                    # Store schema information if we don't have it yet
                    if baseline_schema is None:
                        baseline_schema = baseline_df.schema
                        # Extract column types from the schema
                        for col_name in baseline_df.columns:
                            baseline_column_types[col_name] = baseline_df.schema[col_name]
                        logger.info(f"Established baseline schema with {len(baseline_column_types)} columns")
                    
                    # For baseline data, set the "upgrade" column to "baseline" if it exists
                    if "upgrade" in baseline_df.columns:
                        baseline_df = baseline_df.with_columns(pl.lit("baseline").alias("upgrade"))
                    
                    # Save for later combining
                    baseline_dataframes.append(baseline_df)
            except Exception as e:
                logger.error(f"Error processing baseline for {state}, county {county}: {str(e)}")
    
    if baseline_schema is None:
        logger.error("Could not establish baseline schema from any county data")
        return None
        
    logger.info(f"Baseline schema established with {len(baseline_column_types)} columns")
    
    # Step 2: Process all upgrade data with type consistency
    for state, counties in STATES_COUNTIES.items():
        for county in counties:
            try:
                # Process each upgrade ID
                for upgrade_id in upgrade_ids_to_process:
                    upgrade_df = download_and_process_parquet(state, county, upgrade_id, COLUMNS, county_mapping)
                    
                    if upgrade_df is not None and not upgrade_df.is_empty():
                        # Check for type consistency
                        inconsistent_columns = []
                        for col_name in upgrade_df.columns:
                            if col_name in baseline_column_types:
                                baseline_type = baseline_column_types[col_name]
                                upgrade_type = upgrade_df.schema[col_name]
                                
                                if baseline_type != upgrade_type:
                                    inconsistent_columns.append((col_name, upgrade_type, baseline_type))
                        
                        # Attempt to fix inconsistent column types
                        if inconsistent_columns:
                            logger.warning(f"Found {len(inconsistent_columns)} columns with inconsistent types in {state}, {county}, upgrade {upgrade_id}")
                            
                            # Fix column types to match baseline
                            for col_name, current_type, target_type in inconsistent_columns:
                                try:
                                    logger.info(f"Converting column '{col_name}' from {current_type} to {target_type}")
                                    upgrade_df = upgrade_df.with_columns(
                                        pl.col(col_name).cast(target_type, strict=False)
                                    )
                                except Exception as cast_error:
                                    logger.warning(f"Could not convert column '{col_name}': {str(cast_error)}")
                                    
                                    # Last resort: if conversion fails, force type change even if it means nullifying values
                                    try:
                                        logger.info(f"Forcing conversion for '{col_name}' - may result in null values")
                                        
                                        # Method 1: Try with errors="null"
                                        upgrade_df = upgrade_df.with_columns(
                                            pl.col(col_name).cast(target_type, strict=False)
                                        )
                                    except:
                                        # Method 2: Last resort - create a new column of the right type with null values
                                        # and drop the original column
                                        logger.warning(f"Using last resort for '{col_name}': replacing with nulls")
                                        
                                        # Create a temporary column name
                                        temp_col_name = f"{col_name}_temp"
                                        
                                        # Create a new column of the right type with null values
                                        upgrade_df = upgrade_df.with_columns(
                                            pl.lit(None).cast(target_type).alias(temp_col_name)
                                        )
                                        
                                        # Drop the original column and rename the temp column
                                        upgrade_df = upgrade_df.drop(col_name)
                                        upgrade_df = upgrade_df.rename({temp_col_name: col_name})
                        
                        # Replace upgrade ID with measure ID
                        if "upgrade" in upgrade_df.columns and upgrade_id in upgrade_to_measure_mapping:
                            measure_id = upgrade_to_measure_mapping[upgrade_id]
                            upgrade_df = upgrade_df.with_columns(pl.lit(measure_id).alias("upgrade"))
                        
                        upgrade_dataframes.append(upgrade_df)
                            
            except Exception as e:
                logger.error(f"Error processing upgrades for {state}, county {county}: {str(e)}")
    
    # Combine baseline and upgrade dataframes
    all_dataframes = baseline_dataframes + upgrade_dataframes
    
    if not all_dataframes:
        logger.error("No data could be processed from any county")
        return None
    
    logger.info(f"Concatenating {len(all_dataframes)} dataframes ({len(baseline_dataframes)} baseline, {len(upgrade_dataframes)} upgrade)")
    
    try:
        # Use safe polars concat with schema enforcement
        combined_data = pl.concat(
            all_dataframes,
            how="diagonal_relaxed"  
        )
        
        logger.info(f"Successfully combined data with {len(combined_data)} rows")
    except Exception as concat_error:
        logger.error(f"Error in polars concatenation: {str(concat_error)}")
        
        # Fall back to pandas for concatenation if polars fails
        logger.info("Falling back to pandas for concatenation...")
        
        try:
            # Convert polars dataframes to pandas
            pandas_dfs = []
            for df in all_dataframes:
                # Apply the baseline schema to each dataframe before conversion
                for col_name in df.columns:
                    if col_name in baseline_column_types:
                        target_type = baseline_column_types[col_name]
                        try:
                            # Try to cast to target type
                            df = df.with_columns(pl.col(col_name).cast(target_type, strict=False))
                        except:
                            logger.warning(f"Failed to cast column '{col_name}' to baseline type before pandas conversion")
                
                pandas_dfs.append(df.to_pandas())
            
            # Concatenate with pandas
            combined_pandas = pd.concat(pandas_dfs, ignore_index=True)
            
            # Convert back to polars
            combined_data = pl.from_pandas(combined_pandas)
            
            logger.info(f"Successfully combined data via pandas with {len(combined_data)} rows")
        except Exception as pandas_error:
            logger.error(f"Pandas concatenation also failed: {str(pandas_error)}")
            
            # Ultimate fallback: force string conversion for all columns
            logger.warning("Converting all columns to strings as last resort...")
            
            string_dataframes = []
            for df in all_dataframes:
                # Convert all columns to strings
                cast_dict = {col: pl.Utf8 for col in df.columns}
                string_df = df.cast(cast_dict)
                string_dataframes.append(string_df)
            
            # Now try to concatenate again
            combined_data = pl.concat(string_dataframes)
            
            logger.info(f"Successfully combined string-converted data with {len(combined_data)} rows")
    
    # Ensure columns are in the correct order and only include requested columns
    columns = list(COLUMNS)
    
    # Add county name at the right position if it's available
    if "in.county_name" in combined_data.columns:
        # Find the position to insert the county name 
        try:
            insert_pos = columns.index("in.floor_area_category")
            
            # Insert county name at the right position if it's not already there
            if "in.county_name" not in columns:
                columns.insert(insert_pos, "in.county_name")
        except ValueError:
            # If in.floor_area_category is not in columns, add county_name at the end
            if "in.county_name" not in columns:
                columns.append("in.county_name")
    
    # Get available columns (intersection of desired columns and what's actually in the data)
    available_columns = [col for col in columns if col in combined_data.columns]
    
    # Reorder columns and select only the ones we need
    combined_data = combined_data.select(available_columns)
    
    logger.info(f"Final combined data has {len(combined_data)} rows and {len(available_columns)} columns")
    
    return combined_data


## 4. Main Exection Section

In this main section logging gets set up (Important: Check logger output!) and for all states and counties, the data is downloaded, 
the desired columns are selected and combined into one dataset. This dataset gets written into a
csv file with the filename as specified in 1.


In [7]:

# ========================
# MAIN EXECUTION
# ========================

def main():
    """Main execution function"""
    logger.info("Starting OEDI Building Stock Data Extraction")
    
    # Validate configuration
    if not STATES_COUNTIES:
        logger.error("No states/counties specified. Please update the STATES_COUNTIES dictionary.")
        return
    
    for state, counties in STATES_COUNTIES.items():
        if not isinstance(state, str) or len(state) != 2:
            logger.error(f"Invalid state code: {state}. Must be a 2-character string.")
            return
        
        if not counties:
            logger.error(f"No counties specified for state {state}")
            return
        
        for county in counties:
            if not isinstance(county, str) or not county.startswith('G'):
                logger.error(f"Invalid county code: {county}. Must be a string starting with 'G'.")
                return
    
    # Process all data
    combined_data = process_all_data()
    
    if combined_data is None:
        logger.error("Failed to process data")
        return
    
    # Export to CSV
    try:
        logger.info(f"Exporting data to {OUTPUT_CSV}")
        combined_data.write_csv(OUTPUT_CSV)
        logger.info(f"Successfully exported {len(combined_data)} rows to {OUTPUT_CSV}")
    except Exception as e:
        logger.error(f"Error exporting to CSV: {str(e)}")
        return
    
    logger.info("OEDI Building Stock Data Extraction completed successfully")

if __name__ == "__main__":
    main()

2025-06-02 12:37:54,162 - INFO - Starting OEDI Building Stock Data Extraction
2025-06-02 12:37:54,163 - INFO - Downloading county lookup table from https://oedi-data-lake.s3.amazonaws.com/nrel-pds-building-stock/end-use-load-profiles-for-us-building-stock/2024/comstock_amy2018_release_2/geographic_information/spatial_tract_lookup_table.csv
C:\Users\fantrett\AppData\Local\Temp\1\ipykernel_17572\715145325.py:22: DtypeWarning: Columns (14) have mixed types. Specify dtype option on import or set low_memory=False.
  county_lookup_df = pd.read_csv(
2025-06-02 12:38:00,373 - INFO - County lookup table loaded with 95847 entries in 6.21 seconds
2025-06-02 12:38:00,379 - INFO - Downloading measure crosswalk from https://oedi-data-lake.s3.amazonaws.com/nrel-pds-building-stock/end-use-load-profiles-for-us-building-stock/2024/comstock_amy2018_release_2/measure_name_crosswalk.csv
2025-06-02 12:38:00,719 - INFO - Measure mapping loaded with 39 entries in 0.34 seconds
2025-06-02 12:38:00,720 - INFO - 